Комірка коду нижче імпортує та налаштовує бібліотеки Python.

In [1]:
import os, requests
from datetime import datetime
import pandas as pd

Реалізуємо функції, які необхідні для завантаження та подальшого оновлення файлів csv з вегетаційним індексом VHI:

In [2]:
# Папка для збереження даних
data_folder = 'vhi_data'
os.makedirs(data_folder, exist_ok=True)

# Список областей (індекси від 1 до 27)
oblasts = list(range(1, 28))

# URL-адреса для завантаження файлів
base_url = 'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean'

#функція, щоб видаляти старі файли, щоб не було декілька для одного і того ж регіону
def remove_old_files(region_index):
    for file in os.listdir(data_folder):
        if file.startswith(f'VHI_{region_index}_') and file.endswith('.csv'):
            os.remove(os.path.join(data_folder, file))


def download_vhi_data(region_index):
    # Формування URL для завантаження
    url = base_url.format(region_index)
    # Поточна дата та час
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = f'VHI_{region_index}_{timestamp}.csv'
    filepath = os.path.join(data_folder, filename)
    
    # Завантаження файлу
    response = requests.get(url)
    if response.status_code == 200:
        remove_old_files(region_index)
        with open(filepath, 'wb') as file:
            file.write(response.content)
        print(f'Файл для області {region_index} збережено: {filename}')
    else:
        print(f'Не вдалося завантажити дані для області {region_index}')

# Функція для довантаження нових даних
def update_data():
    for oblast in oblasts:
        download_vhi_data(oblast)

Щоб оновити дані для всіх областей необхідно запустити код нижче:

In [3]:
update_data()

Файл для області 1 збережено: VHI_1_2025-03-17_14-50-46.csv
Файл для області 2 збережено: VHI_2_2025-03-17_14-50-47.csv
Файл для області 3 збережено: VHI_3_2025-03-17_14-50-48.csv
Файл для області 4 збережено: VHI_4_2025-03-17_14-50-49.csv
Файл для області 5 збережено: VHI_5_2025-03-17_14-50-50.csv
Файл для області 6 збережено: VHI_6_2025-03-17_14-50-51.csv
Файл для області 7 збережено: VHI_7_2025-03-17_14-50-52.csv
Файл для області 8 збережено: VHI_8_2025-03-17_14-50-53.csv
Файл для області 9 збережено: VHI_9_2025-03-17_14-50-54.csv
Файл для області 10 збережено: VHI_10_2025-03-17_14-50-55.csv
Файл для області 11 збережено: VHI_11_2025-03-17_14-50-56.csv
Файл для області 12 збережено: VHI_12_2025-03-17_14-50-57.csv
Файл для області 13 збережено: VHI_13_2025-03-17_14-50-58.csv
Файл для області 14 збережено: VHI_14_2025-03-17_14-51-01.csv
Файл для області 15 збережено: VHI_15_2025-03-17_14-51-02.csv
Файл для області 16 збережено: VHI_16_2025-03-17_14-51-03.csv
Файл для області 17 збереж

Реалізуємо також функції, щоб переглянути вміст нашого файлу

In [4]:
def clean_csv_file(filepath):
    # Очищує файл від зайвих ком у кінці рядків та інших артефактів
    with open(filepath, "r", encoding="utf-8") as file:
        lines = [line.replace("</pre></tt>", "").strip(",\n") for line in file]
    with open(filepath, "w", encoding="utf-8") as file:
        file.write("\n".join(lines))

def load_vhi_data(directory):
    # Зчитує всі CSV-файли з директорії в один DataFrame
    all_data = []

    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            clean_csv_file(filepath)  # Очищуємо файл перед обробкою

            with open(filepath, "r", encoding="utf-8") as file:
                first_line = file.readline().strip()
                region_name = first_line.split(":")[-1].split(",")[0].strip()

            df = pd.read_csv(filepath, skiprows=2, header=0, usecols=[0, 1, 2, 3, 4, 5, 6], skip_blank_lines=True)
            df.columns = ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI"]

            region_index = filename.split("_")[1]
            df["Region"] = int(region_index)
            df["Region_Name"] = region_name

            all_data.append(df)

    df = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()
    df = df.dropna(subset=["Year"])  # Видаляємо рядки з NaN у Year
    return df

# Використання
data_folder = 'vhi_data'
vhi_df = load_vhi_data(data_folder)

Перевіримо структуру "голови":

In [5]:
vhi_df.head()

,Year,Week,SMN,SMT,VCI,TCI,VHI,Region,Region_Name
0,1982,2,0.063,261.53,55.89,38.20,47.04,10,Khmel'nyts'kyy
1,1982,3,0.063,263.45,57.30,32.69,44.99,10,Khmel'nyts'kyy
2,1982,4,0.061,265.10,53.96,28.62,41.29,10,Khmel'nyts'kyy
3,1982,5,0.058,266.42,46.87,28.57,37.72,10,Khmel'nyts'kyy
4,1982,6,0.056,267.47,39.55,30.27,34.91,10,Khmel'nyts'kyy


Та "хвоста":

In [6]:
vhi_df.tail()

,Year,Week,SMN,SMT,VCI,TCI,VHI,Region,Region_Name
60340,2024,48,0.135,278.17,55.23,11.86,33.55,9,Kherson
60341,2024,49,0.133,277.08,57.71,10.86,34.29,9,Kherson
60342,2024,50,0.130,276.49,59.45,8.68,34.07,9,Kherson
60343,2024,51,0.128,276.45,62.53,5.55,34.04,9,Kherson
60344,2024,52,0.129,276.48,66.13,3.71,34.92,9,Kherson


Дізнаємось як регіони пронумеровані зараз:

In [7]:
directory = 'vhi_data'

# Сортуємо за числовими індексами, щоб вивід був впорядкованим
file_list = sorted(os.listdir(directory), key=lambda x: int(x.split("_")[1]))
print("Назва регіону та індекс:")
for filename in file_list:
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)

        with open(filepath, "r", encoding="utf-8") as file:
            first_line = file.readline().strip()
            region_name = first_line.split(":")[-1].split(",")[0].strip()
            region_index = filename.split("_")[1]
            print(f"{region_name} - {region_index}")
    


Назва регіону та індекс:
Cherkasy - 1
Chernihiv - 2
Chernivtsi - 3
Crimea - 4
Dnipropetrovs'k - 5
Donets'k - 6
Ivano-Frankivs'k - 7
Kharkiv - 8
Kherson - 9
Khmel'nyts'kyy - 10
Kiev - 11
Kiev City - 12
Kirovohrad - 13
Luhans'k - 14
L'viv - 15
Mykolayiv - 16
Odessa - 17
Poltava - 18
Rivne - 19
Sevastopol' - 20
Sumy - 21
Ternopil' - 22
Transcarpathia - 23
Vinnytsya - 24
Volyn - 25
Zaporizhzhya - 26
Zhytomyr - 27


Тепер замінимо індекси регіонів відповідно до завдання:

Cherkasy - 1 = 22

Chernihiv - 2 = 24

Chernivtsi - 3 = 23

Crimea - 4 = 25

Dnipropetrovs'k - 5 = 3

Donets'k - 6 = 4

Ivano-Frankivs'k - 7 = 8

Kharkiv - 8 = 19

Kherson - 9 = 20

Khmel'nyts'kyy - 10 = 21

Kiev - 11 = 9

Kiev City - 12 = 26

Kirovohrad - 13 = 10

Luhans'k - 14 = 11

L'viv - 15 = 12

Mykolayiv - 16 = 13

Odessa - 17 = 14

Poltava - 18 = 15

Rivne - 19 = 16

Sevastopol' - 20 = 27

Sumy - 21 = 17

Ternopil' - 22 = 18

Transcarpathia - 23 = 6

Vinnytsya - 24 = 1

Volyn - 25 = 2

Zaporizhzhya - 26 = 7

Zhytomyr - 27 = 5

Присвоїмо індекси 26 для Kyiv City та 27 для Sevastopol', оскільки їх номерів немає у наданій таблиці регіонів за українським алфавітом, а також вони не є регіонами, тому, на мою думку, доцільно їх відокремити.


In [8]:
def update_region_indices(df):
    # Оновлює індекси областей у відповідності до українського порядку
    region_mapping = {
    1:22, 2:24, 3:23, 4:25, 5:3, 6:4, 7:8, 8:19, 9:20, 10:21, 11:9, 12:26, 
    13:10, 14:11, 15:12, 16:13, 17: 14, 18:15, 19:16, 20:27, 21:17, 22:18, 
    23:6, 24:1, 25:2, 26:7, 27:5
    }

    df["Region"] = df["Region"].map(region_mapping)
    return df

# Використання
vhi_df = update_region_indices(vhi_df)

Перевіримо:

In [9]:
vhi_df.head()

,Year,Week,SMN,SMT,VCI,TCI,VHI,Region,Region_Name
0,1982,2,0.063,261.53,55.89,38.20,47.04,21,Khmel'nyts'kyy
1,1982,3,0.063,263.45,57.30,32.69,44.99,21,Khmel'nyts'kyy
2,1982,4,0.061,265.10,53.96,28.62,41.29,21,Khmel'nyts'kyy
3,1982,5,0.058,266.42,46.87,28.57,37.72,21,Khmel'nyts'kyy
4,1982,6,0.056,267.47,39.55,30.27,34.91,21,Khmel'nyts'kyy


In [10]:
vhi_df.tail()

,Year,Week,SMN,SMT,VCI,TCI,VHI,Region,Region_Name
60340,2024,48,0.135,278.17,55.23,11.86,33.55,20,Kherson
60341,2024,49,0.133,277.08,57.71,10.86,34.29,20,Kherson
60342,2024,50,0.130,276.49,59.45,8.68,34.07,20,Kherson
60343,2024,51,0.128,276.45,62.53,5.55,34.04,20,Kherson
60344,2024,52,0.129,276.48,66.13,3.71,34.92,20,Kherson


Тепер, коли ми змогли відкрити очистити та відформатувати наші датафрейми, можемо реалізувати процедури для формування вибірок наступного виду:

1. Ряд VHI для області за вказаний рік:

In [11]:
def vhi_series_by_region_year(df, region_name, year):
    return df[(df['Region_Name'] == region_name) & (df['Year'] == year)][['Week', 'VHI']]

Використання (на прикладі Львівської за 1991):

In [12]:
vhi_series_by_region_year(vhi_df, "L'viv", 1991)

,Week,VHI
11642,1,36.79
11643,2,39.36
11644,3,41.00
11645,4,42.04
11646,5,42.66
11647,6,42.54
11648,7,40.54
11649,8,37.09
11650,9,34.10
11651,10,31.31


2. Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани

In [13]:
# max_vhi - максимальне значення
# min_vhi - мінімальне значення
# mean_vhi - середнє значення
# max_vhi - медіана
def vhi_extremes(df, regions, years):
    subset = df[df['Region_Name'].isin(regions) & df['Year'].isin(years)]
    min_vhi = subset.groupby(['Region_Name', 'Year'])['VHI'].min().reset_index().rename(columns={'VHI': 'Min VHI'})
    max_vhi = subset.groupby(['Region_Name', 'Year'])['VHI'].max().reset_index().rename(columns={'VHI': 'Max VHI'})
    mean_vhi = subset.groupby(['Region_Name', 'Year'])['VHI'].mean().reset_index().rename(columns={'VHI': 'Mean VHI'})
    median_vhi = subset.groupby(['Region_Name', 'Year'])['VHI'].median().reset_index().rename(columns={'VHI': 'Median VHI'})
    return min_vhi, max_vhi, mean_vhi, median_vhi

Приклад використання (на прикладі Львівської за 1982 та 2024):

In [14]:
min_vhi, max_vhi, mean_vhi, median_vhi = vhi_extremes(vhi_df, ["L'viv"], [1982, 2024])
print(median_vhi)

  Region_Name  Year  Median VHI
0       L'viv  1982      37.100
1       L'viv  2024      47.795


3. Ряд VHI за вказаний діапазон років для вказаних областей;

In [15]:
def vhi_series_by_year_range(df, regions, start_year, end_year):
    return df[(df['Region_Name'].isin(regions)) & (df['Year'].between(start_year, end_year))][['Year', 'Week', 'Region_Name', 'VHI']]

Приклад використання (на прикладі Херсонської та Київської з 1991 по 2024):

In [16]:
vhi_series_by_year_range(vhi_df, ['Kherson', 'Kiev'], 1991, 2024)

,Year,Week,Region_Name,VHI
2702,1991,1,Kiev,43.45
2703,1991,2,Kiev,45.79
2704,1991,3,Kiev,46.30
2705,1991,4,Kiev,47.50
2706,1991,5,Kiev,43.21
...,...,...,...,...
60340,2024,48,Kherson,33.55
60341,2024,49,Kherson,34.29
60342,2024,50,Kherson,34.07
60343,2024,51,Kherson,34.04


4. Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25).
Повернути роки, назви областей з екстремальними посухами та значення VHI;

Реалізуємо функцію для виконання завдання. Також ігноруватимемо VHI < 0 (переважно -1), оскільки це вочевидь позначка про відсутність значення. А також вважатимемо, що областей 27.

In [17]:
def find_extreme_drought_years(vhi_df, min_reg_perc, threshold=15):
    min_regions = min_reg_perc * 27/100
    # Фільтруємо дані, залишаючи тільки ті записи, де VHI < threshold
    extreme_droughts = vhi_df[(vhi_df['VHI'] < threshold) & (vhi_df['VHI'] >= 0)]
    
    # Групуємо дані за роками і підраховуємо кількість унікальних регіонів із посухою
    drought_years = extreme_droughts.groupby('Year')['Region_Name'].nunique()
    
    # Вибираємо роки, де кількість унікальних областей перевищує мінімальний поріг
    critical_years = drought_years[drought_years >= min_regions].index
    
    # Отримуємо список унікальних областей і мінімальний VHI для кожного року
    result = (extreme_droughts[extreme_droughts['Year'].isin(critical_years)]
              .groupby(['Year', 'Region_Name'])['VHI']
              .min()
              .reset_index())
    
    return result



Спробуємо знайти знайти роки, протягом яких екстремальні значення посухи торкнулися більше 10 відсотків (2,7 в теорії) областей по Україні

In [18]:
find_extreme_drought_years(vhi_df, 10)

,Year,Region_Name,VHI
0,2000,Cherkasy,10.68
1,2000,Kharkiv,9.36
2,2000,Kiev,10.60
3,2000,Kiev City,6.49
4,2000,Sevastopol',8.14
5,2000,Vinnytsya,11.25
6,2007,Crimea,13.28
7,2007,Kherson,12.23
8,2007,Mykolayiv,5.94
9,2007,Odessa,5.52
